In [2]:
import torch
import torch.nn.functional as F
import matplotlib.pyplot as plt
%matplotlib inline

In [3]:
words = open('names.txt', 'r').read().splitlines()
len(words)

32033

In [4]:
chars = sorted(list(set(''.join(words))))
stoi = {s:i+1 for i, s in enumerate(chars)}
stoi['.'] = 0
itos = {i:s for s, i in stoi.items()}

In [11]:
# build the dataset
block_size = 3 #context size
X, Y = [], []

for w in words[:3]:
    print(w)
    context = [0] * 3
    for ch in w:
        ix = stoi[ch]
        X.append(context)
        Y.append(ix)
        print(f'{"".join(itos[i] for i in context)} ----> {itos[ix]}')
        context = context[1:] + [ix] #update context

X = torch.tensor(X)
Y = torch.tensor(Y)

emma
... ----> e
..e ----> m
.em ----> m
emm ----> a
olivia
... ----> o
..o ----> l
.ol ----> i
oli ----> v
liv ----> i
ivi ----> a
ava
... ----> a
..a ----> v
.av ----> a


In [15]:
# Implementing Embedding matrix C (embedding lookup table)

C = torch.randn((27,2)) # In this example we reduce the 26 aphabetic letters + "." into a 2 dimensional space

emb = C[X] #embedding X

emb.shape

torch.Size([13, 3, 2])